# Portfolio Optimization in Python 101 - SciPy edition

## Setup

In [1]:
import requests
import pandas as pd
import numpy as np
import scipy.optimize as sco
import quantstats as qs

# api key
from api_keys import FMP_API_KEY

## Downloading data

In [2]:
FAANG_TICKERS = ["META", "AAPL", "AMZN", "NFLX", "GOOGL"]
START_DATE = "2023-01-01"

In [3]:
def get_adj_close_price(symbol, start_date):
    hist_price_url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?from={start_date}&apikey={FMP_API_KEY}"
    r_json = requests.get(hist_price_url).json()
    df = pd.DataFrame(r_json["historical"]).set_index("date").sort_index()
    df.index = pd.to_datetime(df.index)
    return df[["adjClose"]].rename(columns={"adjClose": symbol})

In [4]:
price_df_list = []
for ticker in FAANG_TICKERS:
    price_df_list.append(get_adj_close_price(ticker, START_DATE))
prices_df = price_df_list[0].join(price_df_list[1:])
prices_df

,META,AAPL,AMZN,NFLX,GOOGL
date,,,,,
2023-01-03,124.61,124.22,85.82,294.950012,89.12
2023-01-04,127.24,125.50,85.14,309.410004,88.08
2023-01-05,126.81,124.17,83.12,309.700012,86.20
2023-01-06,129.88,128.74,86.08,315.549988,87.34
2023-01-09,129.33,129.26,87.36,315.170013,88.02
...,...,...,...,...,...
2024-04-26,443.29,169.30,179.62,561.230000,171.95
2024-04-29,432.62,173.50,180.96,559.490000,166.15
2024-04-30,430.17,170.33,175.00,550.640000,162.78


In [5]:
returns_df = prices_df.pct_change().dropna()
returns_df

,META,AAPL,AMZN,NFLX,GOOGL
date,,,,,
2023-01-04,0.021106,0.010304,-0.007924,0.049025,-0.011670
2023-01-05,-0.003379,-0.010598,-0.023726,0.000937,-0.021344
2023-01-06,0.024209,0.036804,0.035611,0.018889,0.013225
2023-01-09,-0.004235,0.004039,0.014870,-0.001204,0.007786
2023-01-10,0.027217,0.004487,0.028732,0.039249,0.004544
...,...,...,...,...,...
2024-04-26,0.004327,-0.003473,0.034260,-0.006321,0.102244
2024-04-29,-0.024070,0.024808,0.007460,-0.003100,-0.033731
2024-04-30,-0.005663,-0.018271,-0.032935,-0.015818,-0.020283


## Portfolio Optimization

In [6]:
# Calculate the annualized expected returns and the covariance matrix
avg_returns = returns_df.mean() * 252
cov_mat = returns_df.cov() * 252

# Define the function to find the portfolio volatility using the weights and the covariance matrix
def get_portfolio_volatility(weights, cov_mat):
    return np.sqrt(np.dot(weights.T, np.dot(cov_mat, weights)))

# Define the number of assets
n_assets = len(avg_returns)

# Define the bounds - the weights can be between 0 and 1
bounds = tuple((0, 1) for asset in range(n_assets))

# Define the initial guess - the equally weighted portfolio
initial_guess = n_assets * [1.0 / n_assets]

# Define the constraint - all weights must add up to 1
constr = {"type": "eq", "fun": lambda x: np.sum(x) - 1}

# Find the minimum volatility portfolio
min_vol_portf = sco.minimize(
    get_portfolio_volatility,
    x0=initial_guess,
    args=cov_mat,
    method="SLSQP",
    constraints=constr,
    bounds=bounds,
)

# Store the portfolio weights
min_vol_portf_weights = pd.Series(min_vol_portf.x, index=avg_returns.index).round(2)
min_vol_portf_weights

META     0.00
AAPL     0.74
AMZN     0.11
NFLX     0.07
GOOGL    0.08
dtype: float64

In [7]:
min_vol_portf.fun

0.1962945144198569

In [8]:
def get_neg_sharpe_ratio(weights, avg_rtns, cov_mat, rf_rate):
    portf_returns = np.sum(avg_rtns * weights)
    portf_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_mat, weights)))
    portf_sharpe_ratio = (portf_returns - rf_rate) / portf_volatility
    return -portf_sharpe_ratio

In [9]:
RF_RATE = 0
args = (avg_returns, cov_mat, RF_RATE)

max_sharpe_portf = sco.minimize(
    get_neg_sharpe_ratio,
    x0=initial_guess,
    args=args,
    method="SLSQP",
    bounds=bounds,
    constraints=constr,
)

# Store the portfolio weights
max_sharpe_portf_weights = pd.Series(max_sharpe_portf.x, index=avg_returns.index).round(2)
max_sharpe_portf_weights

META     0.50
AAPL     0.00
AMZN     0.17
NFLX     0.21
GOOGL    0.11
dtype: float64

In [10]:
max_sharpe_portf.fun

-2.6415646355541904

## Evaluating the Portfolios

In [14]:
min_vol_portf_returns = returns_df.dot(min_vol_portf_weights)
qs.reports.metrics(min_vol_portf_returns, benchmark="SPY", mode="basic", rf=0)

[*********************100%%**********************]  1 of 1 completed

                    Benchmark (SPY)    Strategy
------------------  -----------------  ----------
Start Period        2023-01-04         2023-01-04
End Period          2024-05-02         2024-05-02
Risk-Free Rate      0.0%               0.0%
Time in Market      100.0%             100.0%

Cumulative Return   31.6%              53.66%
CAGR﹪              15.37%             25.06%

Sharpe              1.68               1.75
Prob. Sharpe Ratio  97.32%             97.87%
Sortino             2.57               2.73
Sortino/√2          1.82               1.93
Omega               1.33               1.33

Max Drawdown        -10.29%            -13.15%
Longest DD Days     122                109

Gain/Pain Ratio     0.31               0.33
Gain/Pain (1M)      1.86               2.77

Payoff Ratio        1.06               1.03
Profit Factor       1.31               1.33
Common Sense Ratio  1.34               1.46
CPC Index           0.76               0.77
Tail Ratio          1.02               1


/Users/elewinson/Documents/private/medium_articles/venv/lib/python3.11/site-packages/quantstats/stats.py:510: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  returns = _utils._prepare_returns(returns, rf).resample(resolution).sum()


In [15]:
max_sharpe_portf_returns = returns_df.dot(max_sharpe_portf_weights)
qs.reports.metrics(max_sharpe_portf_returns, benchmark="SPY", mode="basic", rf=0)

[*********************100%%**********************]  1 of 1 completed

                    Benchmark (SPY)    Strategy
------------------  -----------------  ----------
Start Period        2023-01-04         2023-01-04
End Period          2024-05-02         2024-05-02
Risk-Free Rate      0.0%               0.0%
Time in Market      100.0%             100.0%

Cumulative Return   31.6%              167.27%
CAGR﹪              15.37%             66.84%

Sharpe              1.68               2.6
Prob. Sharpe Ratio  97.32%             99.96%
Sortino             2.57               4.85
Sortino/√2          1.82               3.43
Omega               1.6                1.6

Max Drawdown        -10.29%            -12.51%
Longest DD Days     122                91

Gain/Pain Ratio     0.31               0.6
Gain/Pain (1M)      1.86               7.73

Payoff Ratio        1.07               1.2
Profit Factor       1.31               1.6
Common Sense Ratio  1.34               1.82
CPC Index           0.77               1.08
Tail Ratio          1.02               1.13
O


/Users/elewinson/Documents/private/medium_articles/venv/lib/python3.11/site-packages/quantstats/stats.py:510: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  returns = _utils._prepare_returns(returns, rf).resample(resolution).sum()
